In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
cases_base = spark.sparkContext.textFile("/public/trendytech/covid19/cases")

In [4]:
states_base = spark.sparkContext.textFile("/public/trendytech/covid19/states")

In [7]:
mapped1_rdd = cases_base.map(lambda x : (x.split(",")[1],(int(x.split(",")[2]))))

In [8]:
mapped1_rdd.take(5)

[('AP', 2), ('AP', 2), ('HP', 2), ('HP', 2), ('AS', 2)]

In [12]:
reduced1_rdd = mapped1_rdd.reduceByKey(lambda x,y : x+y).sortBy(lambda x : x[1], False)

In [13]:
reduced1_rdd.take(10)

[('WA', 1701),
 ('GA', 1017),
 ('MH', 730),
 ('MI', 61),
 ('CA', 53),
 ('GJ', 35),
 ('BR', 23),
 ('JH', 13),
 ('CG', 8),
 ('RI', 6)]

In [18]:
icu_count = cases_base.map(lambda x : (int(x.split(",")[7]))).sum()

In [19]:
print(icu_count)

1344


In [20]:
mapped3_rdd = cases_base.map(lambda x : (x.split(",")[1],(int(x.split(",")[11]))))

In [21]:
reduced3_result = mapped3_rdd.reduceByKey(lambda x, y: x+y).sortBy(lambda x : x[1], False)

In [22]:
reduced3_result.take(15)

[('WA', 451),
 ('MH', 165),
 ('MI', 101),
 ('GA', 87),
 ('AP', 84),
 ('RI', 72),
 ('BR', 68),
 ('JH', 50),
 ('KA', 43),
 ('AZ', 38),
 ('AS', 30),
 ('GJ', 27),
 ('CA', 23),
 ('HR', 20),
 ('HP', 19)]

In [23]:
mapped4_rdd = cases_base.map(lambda x : (x.split(",")[1],(int(x.split(",")[23]))))

In [25]:
reduced4_result = mapped4_rdd.reduceByKey(lambda x, y: x+y).sortBy(lambda x : x[1])

In [26]:
reduced4_result.take(5)

[('AS', 9), ('JH', 10), ('CG', 31), ('HR', 32), ('HP', 37)]

In [28]:
mapped5_rdd = cases_base.map(lambda x : (int(x.split(",")[5]))).sum()

In [29]:
print(mapped5_rdd)

1319


In [41]:
mapped6_rdd = states_base.map(lambda x : (x.split(",")[0],(x.split(",")[5],int(x.split(",")[8]))))

In [42]:
mapped6_case_rdd = cases_base.map(lambda x : (x.split(",")[1],(int(x.split(",")[28]))))

In [43]:
total6_case = mapped6_case_rdd.reduceByKey(lambda x,y : x+y)

In [44]:
total6_case.take(5)

[('AP', 4), ('HP', 4), ('AS', 2), ('CG', 8), ('BR', 23)]

In [45]:
mapped6_rdd.take(4)

[('HP', ('@HPCovid', 53)),
 ('AS', ('@ASCovid', 6)),
 ('HR', ('@HRCovid', 9)),
 ('KA', ('@KACovid', 53))]

In [46]:
joined6_rdd = total6_case.join(mapped6_rdd)

In [47]:
joined6_rdd.take(5)

[('MH', (730, ('@MHCovid', 26))),
 ('AS', (2, ('@ASCovid', 6))),
 ('GJ', (35, ('@GJCovid', 44))),
 ('HR', (2, ('@HRCovid', 9))),
 ('KA', (5, ('@KACovid', 53)))]

In [48]:
final6_rdd = joined6_rdd.sortBy(lambda x: x[1][0],False)

In [49]:
final6_rdd.take(10)

[('WA', (2100, ('@WACovid', 44))),
 ('GA', (1034, ('@GACovid', 44))),
 ('MH', (730, ('@MHCovid', 26))),
 ('CA', (515, ('@CACovid', 4))),
 ('MI', (61, ('@MICovid', 53))),
 ('GJ', (35, ('@GJCovid', 44))),
 ('AZ', (34, ('@AZCovid', 53))),
 ('BR', (23, ('@BRCovid', 53))),
 ('RI', (16, ('@RICovid', 26))),
 ('JH', (13, ('@JHCovid', 53)))]

In [50]:
spark.stop()